In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch import optim
import random
import pandas as pd
from sentencepiece import SentencePieceProcessor
from model import *

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [3]:
# INPUT_SIZE = 10000  # Size of the English vocabulary
# OUTPUT_SIZE = 10000  # Size of the Nepali vocabulary
# EMBED_SIZE = 256
# HIDDEN_SIZE = 512
# N_LAYERS = 1
# DROPOUT = 0.5
# BATCH_SIZE = 4
# LEARNING_RATE = 0.001
# EPOCHS = 10
# TEACHER_FORCING_RATIO = 0.5
INPUT_SIZE = 8000  # Matches SentencePiece tokenizer vocabulary size
OUTPUT_SIZE = 8000  # Matches SentencePiece tokenizer vocabulary size
EMBED_SIZE = 128  # Reduce for memory constraints
HIDDEN_SIZE = 256  # Reduce for memory constraints
N_LAYERS = 1  # Single layer GRU
DROPOUT = 0.3  # Prevent overfitting
BATCH_SIZE = 2  # Reduce for memory constraints
LEARNING_RATE = 0.001  # Standard learning rate
EPOCHS = 10  # Initial testing
TEACHER_FORCING_RATIO = 0.5  # Balanced teacher forcing


In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

In [5]:
class TranslationDataset(Dataset):
    def __init__(self, cleaned_file_path):
        self.pairs = self.load_data(cleaned_file_path)

    def load_data(self, cleaned_file_path):
        df = pd.read_excel(cleaned_file_path)
        df = df.dropna()
        english_sentences = df['english_sent'].tolist()
        nepali_sentences = df['nepali_sent'].tolist()

        english_tokenizer = SentencePieceProcessor(model_file='english_sp.model')
        nepali_tokenizer = SentencePieceProcessor(model_file='nepali_sp.model')

        pairs = []
        for english_sentence, nepali_sentence in zip(english_sentences, nepali_sentences):
            english_indices = self.process_sentence(english_sentence, english_tokenizer)
            nepali_indices = self.process_sentence(nepali_sentence, nepali_tokenizer)
            pairs.append((english_indices, nepali_indices))
        
        return pairs

    def process_sentence(self, sentence, tokenizer):
        tokens = tokenizer.encode(sentence, out_type=int)
        return [1] + tokens + [2]

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        return self.pairs[idx]


In [6]:
def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_lens = [len(src) for src in src_batch]
    trg_lens = [len(trg) for trg in trg_batch]
    src_padded = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq) for seq in src_batch], padding_value=0)
    trg_padded = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq) for seq in trg_batch], padding_value=0)
    return src_padded, trg_padded, src_lens, trg_lens


In [7]:

# Initialize model
encoder = Encoder(INPUT_SIZE, EMBED_SIZE, HIDDEN_SIZE, N_LAYERS, DROPOUT).to(DEVICE)
decoder = Decoder(EMBED_SIZE, HIDDEN_SIZE, OUTPUT_SIZE, N_LAYERS, DROPOUT).to(DEVICE)
model = Seq2Seq(encoder, decoder).to(DEVICE)

/media/epein5/Data/English-to-Nepali-Translator/envenv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


In [8]:
# Optimizer and Loss
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [9]:
# # Training loop
# def train_model(dataset):
    
#     dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
#     print("Data loaded...")
#     print("Training started...")
#     model.train()
#     for epoch in range(EPOCHS):
#         epoch_loss = 0
#         for src, trg, _, _ in dataloader:
#             src, trg = src.to(DEVICE), trg.to(DEVICE)
#             optimizer.zero_grad()
#             output = model(src, trg, TEACHER_FORCING_RATIO)
#             output_dim = output.shape[-1]
#             output = output[1:].view(-1, output_dim)
#             trg = trg[1:].view(-1)
#             loss = criterion(output, trg)
#             loss.backward()
#             optimizer.step()
#             # print(f"Loss: {loss.item()}")
#             epoch_loss += loss.item()
#         print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader):.4f}")

from torch.utils.tensorboard import SummaryWriter

# Initialize TensorBoard writer
writer = SummaryWriter(log_dir="runs/translation_experiment")

def train_model(dataset):
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
    print("Data loaded...")
    print("Training started...")
    model.train()
    for epoch in range(EPOCHS):
        epoch_loss = 0
        for src, trg, _, _ in dataloader:
            src, trg = src.to(DEVICE), trg.to(DEVICE)
            optimizer.zero_grad()
            output = model(src, trg, TEACHER_FORCING_RATIO)
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        # Log loss to TensorBoard
        avg_loss = epoch_loss / len(dataloader)
        writer.add_scalar("Loss/Train", avg_loss, epoch)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")
    
    writer.close()


In [10]:
dataset = TranslationDataset("Dataset/english-nepali-cleaned.xlsx")

In [11]:
train_model(dataset)

Data loaded...
Training started...
Epoch 1, Loss: 5.7407
Epoch 2, Loss: 5.1239
Epoch 3, Loss: 4.9306
Epoch 4, Loss: 4.8292
Epoch 5, Loss: 4.7663
Epoch 6, Loss: 4.7245
Epoch 7, Loss: 4.6938
Epoch 8, Loss: 4.6749
Epoch 9, Loss: 4.6627
Epoch 10, Loss: 5.1701


In [12]:
# Save model
torch.save(model.state_dict(), "seq2seq_model.pth")

In [18]:
def translate_sentence(sentence, model, english_tokenizer, nepali_tokenizer, max_len=50):
    """
    Translates an English sentence to Nepali.

    Args:
    - sentence (str): The input English sentence to translate.
    - model (torch.nn.Module): The trained translation model.
    - english_tokenizer (sentencepiece.SentencePieceProcessor): Tokenizer for English.
    - nepali_tokenizer (sentencepiece.SentencePieceProcessor): Tokenizer for Nepali.
    - max_len (int): Maximum length for the translated sentence.

    Returns:
    - str: The translated sentence in Nepali.
    """
    model.eval()

    # Tokenize and encode the input sentence
    src_tokens = [english_tokenizer.bos_id()] + english_tokenizer.encode(sentence) + [english_tokenizer.eos_id()]
    src_tensor = torch.LongTensor(src_tokens).unsqueeze(0).to(DEVICE)

    # Prepare the initial target tensor
    trg_tokens = [nepali_tokenizer.bos_id()]
    trg_tensor = torch.LongTensor(trg_tokens).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        for _ in range(max_len):
            # Pass through the model
            output = model(src_tensor, trg_tensor, 0)  # No teacher forcing during inference
            next_token = output.argmax(2)[:, -1].item()

            # Stop if EOS token is generated
            if next_token == nepali_tokenizer.eos_id():
                break

            trg_tensor = torch.cat([trg_tensor, torch.LongTensor([[next_token]]).to(DEVICE)], dim=1)

    # Decode the output tokens to Nepali sentence
    translated_tokens = trg_tensor.squeeze(0).tolist()
    print(translated_tokens)
    translated_sentence = nepali_tokenizer.decode(translated_tokens[1:-1])  # Exclude BOS and EOS tokens

    return translated_sentence


In [19]:
# Load your tokenizers
import sentencepiece as spm
english_tokenizer = spm.SentencePieceProcessor()
english_tokenizer.load("english_sp.model")

nepali_tokenizer = spm.SentencePieceProcessor()
nepali_tokenizer.load("nepali_sp.model")

# Example sentence
english_sentence = "How are you?"

# Translate
nepali_translation = translate_sentence(
    english_sentence, model, english_tokenizer, nepali_tokenizer
)
print("Translated Sentence:", nepali_translation)


[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Translated Sentence:  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 


In [17]:
test_sentence = "This is a test."
tokens = english_tokenizer.encode(test_sentence)
print("Encoded Tokens:", tokens)
decoded_sentence = english_tokenizer.decode(tokens)
print("Decoded Sentence:", decoded_sentence)


Encoded Tokens: [76, 12, 11, 1050, 5]
Decoded Sentence: This is a test.
